In [2]:
import redis
import numpy as np
import pandas as pd

In [25]:
hostname = 'redis-18992.c258.us-east-1-4.ec2.redns.redis-cloud.com'
port = '18992'
password = 'nUoPu0pKGDk95n5zatr1N1DgTDNmnAuG'

In [26]:
r = redis.StrictRedis(host=hostname,
               port= port, password=password)

In [27]:
r.ping()

True

In [16]:
# load the data numpy.zip

In [28]:
data_restored = np.load('face_dataframe_compressed.npz', allow_pickle=True)
x_values = data_restored['arr_0']
columns = data_restored['arr_1']

df = pd.DataFrame(x_values, columns=columns)
df

,Name,Role,Facial Features,cosine
0,Angelina Jolie,student,"[0.43239492, 1.0748208, -0.12874822, -1.833274...",-0.055578
1,Barack_Obama,teacher,"[0.94851226, -0.5582868, 0.09495868, -1.279620...",-0.109103
2,Chris_Evans,student,"[0.7200426, -1.4685223, -0.7779212, -1.460148,...",0.0282
3,Morgan_Freeman,teacher,"[0.48528442, -0.619405, -0.4070305, 1.0686811,...",0.07118
4,Scarlett Johansson,student,"[0.19131908, -0.48838148, -1.690325, 0.4423288...",0.013093


In [7]:
# genereate unique key name + role for storing data in redis
df['name_role'] = df['Name']+'#'+df['Role']

In [8]:
df

,Name,Role,Facial Features,cosine,name_role
0,Angelina Jolie,student,"[0.43239492, 1.0748208, -0.12874822, -1.833274...",-0.055578,Angelina Jolie#student
1,Barack_Obama,teacher,"[0.94851226, -0.5582868, 0.09495868, -1.279620...",-0.109103,Barack_Obama#teacher
2,Chris_Evans,student,"[0.7200426, -1.4685223, -0.7779212, -1.460148,...",0.0282,Chris_Evans#student
3,Morgan_Freeman,teacher,"[0.48528442, -0.619405, -0.4070305, 1.0686811,...",0.07118,Morgan_Freeman#teacher
4,Scarlett Johansson,student,"[0.19131908, -0.48838148, -1.690325, 0.4423288...",0.013093,Scarlett Johansson#student


In [9]:
# convert records to dictionary to store in redis
records = df[['name_role','Facial Features']].to_dict(orient='records')
len(records)

5

## Save data in Redis
Using Redis Hashes
keyname = 'school:register'

In [10]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial Features']
    
    # convert numpy array to bytes
    vector_bytes = vector.tobytes()
    
    # save data to redis cloud
    r.hset(name='school:register',key=name_role,value=vector_bytes)

In [11]:
vector.dtype

dtype('float32')

## Retrive Records from Redis

In [12]:
decoded_dictionary = r.hgetall(name='school:register')
decoded_series = pd.Series(decoded_dictionary)
decoded_series

b'Niral#Student'                 b'C\xd4z\xbf\xf1$\x9e\xbe~\xcd\x7f=\xfd\xf6^\x...
b'Piyush#Student'                b'\x05\x13V\xbf>\x8aT?"Uv\xbf\xfb5\xae\xbft\xc...
b'Harsh#Student'                 b'\xe2:\xb4>\x0fi\xc8\xbe\x1fb\n>\xe1\x93\x92>...
b'Shreyash#Student'              b'\xaak"?\x11\xfa\x1a\xbf\x10\'D\xbd\xbd\xdaQ=...
b'Anshu#Student'                 b'h\x0b\xf2\xbe\\\xb5\x0b\xbf\xe3\xb1\xb8>0+\x...
b'Shivam#Student#Shivam@1234'    b'g\xc5\'?\x94\x1d\x94\xbe\x03\x0f\x00>\xd2\xe...
b'Barack_Obama#teacher'          b'\xb3\xd1r?\xe2\xeb\x0e\xbf\xb2y\xc2=\x9a\xca...
b'Morgan_Freeman#teacher'        b'3w\xf8>S\x91\x1e\xbfMf\xd0\xbe\x8b\xca\x88?\...
b'Chris_Evans#student'           b'\xb6T8?\x8a\xf8\xbb\xbf\xd8%G\xbf!\xe6\xba\x...
b'Shruti#Student'                b'r\xd1\xed\xbd~y|\xbek6\xfe\xbc\x0c|\xd4?\x94...
b'Angelina Jolie#student'        b'\xdeb\xdd>\xba\x93\x89?\x93\xd6\x03\xbe\xba\...
b'Scarlett Johansson#student'    b'&\xe9C>#\r\xfa\xbe\x92\\\xd8\xbf\xedx\xe2>\x...
dtyp

In [13]:
# Convert each element of retrive_series from bytes to 32-bit float NumPy arrays
decoded_series = decoded_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))

# Retrieve the index of retrive_series
index = decoded_series.index

# Print the retrieved index
print(index)

# Convert the index elements from bytes to strings
index = list(map(lambda x: x.decode(), index))



Index([             b'Niral#Student',             b'Piyush#Student',
                    b'Harsh#Student',           b'Shreyash#Student',
                    b'Anshu#Student', b'Shivam#Student#Shivam@1234',
             b'Barack_Obama#teacher',     b'Morgan_Freeman#teacher',
              b'Chris_Evans#student',             b'Shruti#Student',
           b'Angelina Jolie#student', b'Scarlett Johansson#student'],
      dtype='object')


In [14]:
# Print the modified index after decoding
print(index)

['Niral#Student', 'Piyush#Student', 'Harsh#Student', 'Shreyash#Student', 'Anshu#Student', 'Shivam#Student#Shivam@1234', 'Barack_Obama#teacher', 'Morgan_Freeman#teacher', 'Chris_Evans#student', 'Shruti#Student', 'Angelina Jolie#student', 'Scarlett Johansson#student']


In [15]:
decoded_series.index = index

In [16]:
decoded_df = retrive_series.to_frame().reset_index()
decoded_df.columns = ['name_role', 'facial_features']
decoded_df

NameError: name 'retrive_series' is not defined

In [24]:
        decoded_dictionary = r.hgetall(name='school:register')
        decoded_series = pd.Series(decoded_dictionary)
        # Convert each element of retrive_series from bytes to 32-bit float NumPy arrays
        decoded_series = decoded_series.apply(lambda x: np.frombuffer(x, dtype=np.float32))

        # Retrieve the index of retrive_series
        index = decoded_series.index

        # Convert the index elements from bytes to strings
        index = list(map(lambda x: x.decode(), index))
        decoded_series.index = index
        decoded_df = decoded_series.to_frame().reset_index()
        decoded_df.columns = ['name_role', 'facial_features']
        decoded_df.rename(columns={'facial_features': 'Facial Features'}, inplace=True)
        # Split by '#' and create a new DataFrame with two columns 'Name' and 'Role'
        temp_df = decoded_df['name_role'].str.split('#', expand=True)

        # Split each part by '_' and capitalize each word
        temp_df.map(lambda x: ' '.join([word.capitalize() for word in x.split('_')]) if x is not None  else 'N/A')

,0,1,2
0,Niral,Student,N/A
1,Piyush,Student,N/A
2,Harsh,Student,N/A
3,Shreyash,Student,N/A
4,Anshu,Student,N/A
5,Shivam,Student,Shivam@1234
6,Barack Obama,Teacher,N/A
7,Morgan Freeman,Teacher,N/A
8,Chris Evans,Student,N/A
9,Shruti,Student,N/A
